In [3]:
!git clone https://github.com/xinyu1205/recognize-anything.git
!pip install timm transformers fairscale pycocoevalcap
%cd recognize-anything


/home/fuduweiii/IT/Project/python/pipeline-hcm-ai/notebooks/data_extraction/metadata/recognize-anything


In [4]:
import os
import glob
import json
import torch
import numpy as np
from PIL import Image
from ram.models import ram_plus
from ram import get_transform
from tqdm import tqdm

/home/fuduweiii/IT/miniconda3/envs/hcmcai-3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Download CheckPoint

In [35]:
def download_checkpoints(model):
    print('You selected', model)
    if not os.path.exists('pretrained'):
        os.makedirs('pretrained')

    if model == "RAM++":
        ram_weights_path = 'pretrained/ram_swin_large_14m.pth'
        if not os.path.exists(ram_weights_path):
            !wget https://huggingface.co/xinyu1205/recognize-anything-plus-model/resolve/main/ram_plus_swin_large_14m.pth -O pretrained/ram_swin_large_14m.pth
        else:
            print("RAM weights already downloaded!")

model = "RAM++"
download_checkpoints(model)
print(model, 'weights are downloaded!')

You selected RAM++
--2024-08-24 16:25:29--  https://huggingface.co/xinyu1205/recognize-anything-plus-model/resolve/main/ram_plus_swin_large_14m.pth
Resolving huggingface.co (huggingface.co)... 108.157.32.94, 108.157.32.95, 108.157.32.66, ...
Connecting to huggingface.co (huggingface.co)|108.157.32.94|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/1f/c0/1fc0c455d992a58616eaae5f3ce9e34b1d2c49026fede34de9a7a5f3d06373dd/497c178836ba66698ca226c7895317e6e800034be986452dbd2593298d50e87d?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27ram_plus_swin_large_14m.pth%3B+filename%3D%22ram_plus_swin_large_14m.pth%22%3B&Expires=1724750730&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyNDc1MDczMH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzFmL2MwLzFmYzBjNDU1ZDk5MmE1ODYxNmVhYWU1ZjNjZTllMzRiMWQyYzQ5MDI2ZmVkZTM0ZGU5YTdhNWYzZDA2MzczZGQvNDk3YzE3OD

# Parse data path

In [23]:
keyframes_dir = '../Keyframes'
all_keyframe_paths = dict()
for part in sorted(os.listdir(keyframes_dir)):
    all_keyframe_paths[part] =  dict() # L01, L02

for data_part in sorted(all_keyframe_paths.keys()):
    data_part_path = f'{keyframes_dir}/{data_part}'       # ../Keyframes/L01
    video_dirs = sorted(os.listdir(data_part_path))         # ['V001', 'V002', ...]
    video_ids = [video_dir for video_dir in video_dirs] 
    for video_id, video_dir in zip(video_ids, video_dirs):
        keyframe_paths = sorted(glob.glob(f'{data_part_path}/{video_dir}/*.jpg'))
        all_keyframe_paths[data_part][video_id] = keyframe_paths

output_file = '../data/test.json'
os.makedirs(os.path.dirname(output_file), exist_ok=True)
with open(output_file, 'w') as f:
    json.dump(all_keyframe_paths, f, indent=4)  
print(all_keyframe_paths[data_part][video_id])

['../Keyframes/L01/V031/000000.jpg', '../Keyframes/L01/V031/000006.jpg', '../Keyframes/L01/V031/000013.jpg', '../Keyframes/L01/V031/000020.jpg', '../Keyframes/L01/V031/000027.jpg', '../Keyframes/L01/V031/000028.jpg', '../Keyframes/L01/V031/000123.jpg', '../Keyframes/L01/V031/000219.jpg', '../Keyframes/L01/V031/000315.jpg', '../Keyframes/L01/V031/000411.jpg', '../Keyframes/L01/V031/000412.jpg', '../Keyframes/L01/V031/000426.jpg', '../Keyframes/L01/V031/000440.jpg', '../Keyframes/L01/V031/000454.jpg', '../Keyframes/L01/V031/000469.jpg', '../Keyframes/L01/V031/000470.jpg', '../Keyframes/L01/V031/000479.jpg', '../Keyframes/L01/V031/000489.jpg', '../Keyframes/L01/V031/000499.jpg', '../Keyframes/L01/V031/000509.jpg', '../Keyframes/L01/V031/000510.jpg', '../Keyframes/L01/V031/000520.jpg', '../Keyframes/L01/V031/000530.jpg', '../Keyframes/L01/V031/000540.jpg', '../Keyframes/L01/V031/000550.jpg', '../Keyframes/L01/V031/000551.jpg', '../Keyframes/L01/V031/000562.jpg', '../Keyframes/L01/V031/0005

In [24]:
import json
import torch
from PIL import Image
from ram.models import ram_plus
from ram import inference_ram as inference
from ram import get_transform
import os
from tqdm import tqdm

# Configuration
PRETRAINED_MODEL_PATH = 'pretrained/ram_plus_swin_large_14m.pth'
IMAGE_SIZE = 384
INPUT_JSON = '../data/test.json'
OUTPUT_JSON = '../data/keyframe_tags.json'

def main():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    transform = get_transform(image_size=IMAGE_SIZE)

    # Load model
    model = ram_plus(pretrained=PRETRAINED_MODEL_PATH, image_size=IMAGE_SIZE, vit='swin_l')
    model.eval()
    model = model.to(device)

    # Load input JSON
    with open(INPUT_JSON, 'r') as f:
        keyframe_paths = json.load(f)

    # Initialize output dictionary
    output_data = {}

    # Process each keyframe
    for part in tqdm(keyframe_paths, desc="Processing parts"):
        output_data[part] = {}
        for video_id in tqdm(keyframe_paths[part], desc=f"Processing videos in {part}", leave=False):
            output_data[part][video_id] = {}
            for image_path in tqdm(keyframe_paths[part][video_id], desc=f"Processing keyframes in {video_id}", leave=False):
                try:
                    image = transform(Image.open(image_path)).unsqueeze(0).to(device)
                    res = inference(image, model)
                    tags = res[0].split(' | ')
                    output_data[part][video_id][os.path.basename(image_path)] = tags
                except Exception as e:
                    print(f"Error processing {image_path}: {str(e)}")   

    # Save output JSON
    with open(OUTPUT_JSON, 'w') as f:
        json.dump(output_data, f, indent=4)

    print(f"Tag extraction complete. Results saved to {OUTPUT_JSON}")

if __name__ == "__main__":
    main()

--------------
/home/fuduweiii/IT/Project/python/pipeline-hcm-ai/notebooks/data_extraction/metadata/recognize-anything/pretrained/ram_plus_swin_large_14m.pth
--------------
load checkpoint from /home/fuduweiii/IT/Project/python/pipeline-hcm-ai/notebooks/data_extraction/metadata/recognize-anything/pretrained/ram_plus_swin_large_14m.pth
vit: swin_l


Processing parts:   0%|          | 0/1 [00:00<?, ?it/s]


















































































































































































































































































































































































































































































































































































































































































































































































































































































































































































Tag extraction complete. Results saved to ../data/keyframe_tags.json
